## Prompt Generation Demo

In [1]:
import json
import random
from pathlib import Path
from copy import deepcopy

assets_path = Path('./assets')
img_path = './original_images/'
img_id = '2353248'
target_object = 'bus'

### Swap noun case

When constructing SameSwap, we employed additional filtering process to avoid contextual mismatches. 

In [2]:
with open(assets_path / 'open_images_hierarchy.json', 'r') as f:
   class_hier = json.load(f)

# Create list of hierarchical objects from Open Images
def nest(subcategory):
    def _nest(item, level_cat):
        if isinstance(item, list):
            for c in item:
                if isinstance(c, dict):
                    if 'Subcategory' in c.keys():
                        level_cat.append(_nest(c['Subcategory'], []))
                        
                    else:
                        level_cat.append(c['LabelName'])
                else:
                    print('Error')
        return level_cat
    return _nest(subcategory, [])

categories = nest(class_hier['Subcategory'])

# Unwarp Hierarchy into Flattened List & Substitute noun with same category noun
def unwrap_list(categories, all_list):
    def _unwrap_list(lists):
        tmp = []
        for value in lists:
            if isinstance(value, str):
                tmp.append(value.lower())
            elif isinstance(value, list):
                _unwrap_list(value)
        all_list.append(tmp)
        return all_list
    return _unwrap_list(categories)

flattened = unwrap_list(categories, all_list=[])[:-1]

FileNotFoundError: [Errno 2] No such file or directory: 'assets/open_images_hierarchy.json'

In [ ]:
candidates = [idx for idx, tf in enumerate([target_object in _items for _items in flattened]) if tf]

if len(candidates) and len(flattened[candidates[0]])>1:
    sibling_category = deepcopy(flattened[candidates[0]])
    sibling_category.remove(target_object)
    subst = random.choice(sibling_category)
    print(f"Prompt: Change {target_object} to {subst}")
else:
    print("No candidates")

### Swap attr case

In [ ]:
with open(assets_path / 'train_sceneGraphs.json', 'r') as f:
    scene_graph = json.load(f)
with open(assets_path / 'attribute_candidates.json', 'r') as f:
    attr_candidate = json.load(f)
    
in_image_attrs = [v['attributes'] for v in scene_graph[img_id]['objects'].values() if v['name']==target_object]
all_candidates = list(set(attr_candidate[target_object]) - set(in_image_attrs[0]))

if len(all_candidates):
    chosen = random.choice(all_candidates)
    print(f"Prompt: Change {target_object} to {chosen} {target_object}")
else:
    print("No candidates")